# 01 - Import STATS19 Dataset, Merge & Save

## Overview

The UK Department for Transport (DfT) road casualty statistics consist of [three primary datasets](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-accidents-safety-data):
- Collision data: Information about each accident event
- Vehicle data: Details about vehicles involved in accidents
- Casualty data: Information about people injured in accidents

These datasets are linked by common identifiers and can be quite large (approximately 4.5GB combined).

## Memory Challenges

When processing large datasets, loading everything into memory at once can lead to `MemoryError` issues. A solution is to process the data in manageable chunks, focusing on:

1. Reading data incrementally 
2. Applying early filtering to reduce data volume
3. Only loading relevant portions of secondary datasets
4. Writing results incrementally to disk

In [4]:
# Import necessary libraries
import pandas as pd
from pathlib import Path
import os

### Chunked Processing Functions
We define specialized functions to handle large data processing:
- `load_filtered_csv`: loads and filters `.csv` files in chunks.

In [5]:
def load_filtered_csv(path, filter_func=None, dtype=None, chunksize=100_000):
    """
    Load and filter a CSV file in chunks
    
    Parameters:
    path : Path to CSV file
    filter_func : Function to filter rows (optional)
    dtype : Dictionary of column data types
    chunksize : Number of rows to process at once
    """
    chunks = []
    for chunk in pd.read_csv(path, dtype=dtype, chunksize=chunksize, low_memory=False):
        if filter_func is not None:
            chunk = chunk[filter_func(chunk)]
        chunks.append(chunk)
        
    return pd.concat(chunks, ignore_index=True) if chunks else pd.DataFrame()

- `filter_south_yorkshire`: filter function to isolate data for South Yorkshire

In [6]:
def filter_south_yorkshire(df):
    return df['police_force'] == 14

`clean_and_organise_data`: removes redundant columns and orders identifier columns to the front.

In [7]:
def clean_and_organize_data(merged_data):
    """
    Clean up and organize columns in the merged dataset
    
    Parameters:
    merged_data : DataFrame containing the merged data
    
    Returns:
    DataFrame with cleaned and reorganized columns
    """
    print("  Cleaning and organizing columns...")
    
    # List of columns to drop as identified earlier
    columns_to_drop = [
        'accident_year_x', 'accident_year_y',
        'accident_reference_x', 'accident_reference_y'
    ]
    
    # Drop redundant columns
    cleaned_data = merged_data.drop(columns=columns_to_drop, errors='ignore')
    
    # Rename 'vehicle_reference_x' if it exists
    if 'vehicle_reference_x' in cleaned_data.columns:
        cleaned_data = cleaned_data.rename(columns={'vehicle_reference_x': 'vehicle_reference'})
    
    # Reorder columns to bring reference columns to the front
    reference_columns = [col for col in [
        'accident_index', 'accident_year', 'accident_reference', 
        'vehicle_reference', 'casualty_reference'
    ] if col in cleaned_data.columns]
    
    # Identify remaining columns that aren't in reference_columns
    remaining_columns = [col for col in cleaned_data.columns if col not in reference_columns]
    
    # Combine the lists to reorder DataFrame columns
    ordered_columns = reference_columns + remaining_columns
    
    return cleaned_data[ordered_columns]

- `process_in_chunks`: Main processing function that handles the entire workflow. 

In [ ]:
def process_in_chunks(casualty_path, collision_path, vehicle_path, output_path, 
                      filter_func=None, chunksize=50_000, dtype_dict=None):
    """
    Process large datasets in chunks - reading, merging, cleaning and filtering
    
    Parameters:
    casualty_path : Path to casualty CSV
    collision_path : Path to collision CSV
    vehicle_path : Path to vehicle CSV
    output_path : Where to save the final filtered data
    filter_func : Function to filter the final dataset (optional)
    chunksize : Number of rows to process at once
    dtype_dict : Dictionary of column data types
    """
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Process collision data in chunks first (usually smaller than casualty data)
    print("Loading and processing collision data...")
    collision_chunks = pd.read_csv(collision_path, dtype=dtype_dict, 
                                  chunksize=chunksize, low_memory=False)
    
    # Use the first chunk to initialize the output file
    first_chunk = True
    
    for i, collision_chunk in enumerate(collision_chunks):
        print(f"Processing collision chunk {i+1}...")
        
        # If we're filtering by police force, do it early to reduce data volume
        if filter_func is not None:
            collision_chunk = collision_chunk[filter_func(collision_chunk)]
            
            # Skip empty chunks
            if len(collision_chunk) == 0:
                continue
        
        # Get the accident indices in this chunk to filter other datasets
        chunk_accident_indices = set(collision_chunk['accident_index'])
        
        # Load only the casualty data relevant to this chunk's accident indices
        print(f"  Loading relevant casualty data...")
        casualty_chunks = pd.read_csv(casualty_path, dtype=dtype_dict, 
                                     chunksize=chunksize, low_memory=False)
        
        relevant_casualty_data = []
        for casualty_chunk in casualty_chunks:
            filtered_casualty = casualty_chunk[
                casualty_chunk['accident_index'].isin(chunk_accident_indices)]
            
            if not filtered_casualty.empty:
                relevant_casualty_data.append(filtered_casualty)
        
        if not relevant_casualty_data:
            continue
            
        casualty_data = pd.concat(relevant_casualty_data, ignore_index=True)
        
        # Merge casualty with this collision chunk
        print(f"  Merging casualty and collision data...")
        merged_casualty_collision = casualty_data.merge(
            collision_chunk, on="accident_index", how="inner")
        
        # Get the accident indices and vehicle references in this merged chunk
        merge_keys = merged_casualty_collision[['accident_index', 'vehicle_reference']]
        
        # Load only the vehicle data relevant to this chunk
        print(f"  Loading relevant vehicle data...")
        vehicle_chunks = pd.read_csv(vehicle_path, dtype=dtype_dict, 
                                   chunksize=chunksize, low_memory=False)
        
        relevant_vehicle_data = []
        for vehicle_chunk in vehicle_chunks:
            # Create a temporary key for matching
            vehicle_chunk_keys = vehicle_chunk[['accident_index', 'vehicle_reference']]
            # Merge the keys to identify matches
            matches = pd.merge(vehicle_chunk_keys, merge_keys, 
                              on=['accident_index', 'vehicle_reference'],
                              how='inner')
            
            if not matches.empty:
                # Get the matched rows from vehicle chunk
                matched_indices = matches.index
                relevant_vehicle = vehicle_chunk.loc[matched_indices]
                relevant_vehicle_data.append(relevant_vehicle)
        
        if not relevant_vehicle_data:
            continue
            
        vehicle_data = pd.concat(relevant_vehicle_data, ignore_index=True)
        
        # Final merge for this chunk
        print(f"  Creating final merged dataset...")
        final_chunk = merged_casualty_collision.merge(
            vehicle_data, on=["accident_index", "vehicle_reference"], how="inner")
        
        # Clean up the merged chunk using our separated function
        final_chunk = clean_and_organize_data(final_chunk)
        
        # Write to output file
        print(f"  Writing processed data to file...")
        mode = 'w' if first_chunk else 'a'
        header = first_chunk
        final_chunk.to_csv(output_path, mode=mode, header=header, index=False)
        first_chunk = False
        
        # Free memory
        del collision_chunk, casualty_data, vehicle_data, final_chunk
        
    print(f"Processing complete. Data saved to {output_path}")

This function:

1. Processes collision data in chunks
2. For each chunk, applies filtering if specified
3. Identifies accident indices in the current chunk
4. Loads only relevant casualty and vehicle data using these indices
5. Merges the datasets appropriately
6. Cleans and organizes the columns
7. Writes each processed chunk to the output file
8. Frees memory after each chunk is processed

#### Memory Optimization Strategy
Our approach follows these key principles:

1. Early filtering: Apply geographic filtering (South Yorkshire) early to minimize data volume
2. Selective loading: Only load data relevant to the current processing chunk
3. Incremental output: Write results to disk as they're processed rather than accumulating in memory
4. Memory cleanup: Explicitly delete intermediate dataframes after they're no longer needed

#### Data Type Specification
We specify data types for identifier columns to ensure consistent joining:

In [ ]:
# Specify data types for critical columns
dtype_dict = {
    'accident_index': str,
    'accident_year': str, 
    'accident_reference': str,
    'vehicle_reference': str,
    'casualty_reference': str
}

Now that we have the functions defined, we can run the code and process our datasets:

In [ ]:
# File paths
my_dir_path = Path('F:/downloads')
save_path = Path('../data/datasets')
output_file = '../data/STATS19/dft_STATS19_1979_23_SY.csv'

# Process the data in chunks
process_in_chunks(
    casualty_path=my_dir_path/'dft-road-casualty-statistics-casualty-1979-latest-published-year.csv',
    collision_path=my_dir_path/'dft-road-casualty-statistics-collision-1979-latest-published-year.csv',
    vehicle_path=my_dir_path/'dft-road-casualty-statistics-vehicle-1979-latest-published-year.csv',
    output_path=output_file,
    filter_func=filter_south_yorkshire,
    chunksize=50_000,
    dtype_dict=dtype_dict
)

Now that we have imported the datasets, we can merge them to one.

There will be a few extra redundant columns formed due to joining the datasets, which we need to remove.

Now, we can filter the whole dataset for only South Yorkshire data.